## Hadoop

It contains
1. HDFS: File system to manage the storage of data
2. MapReduce: A framework to process data across multiple servers

* Based on GFS

In 2013, Hadoop's MapReduce was divided into 2
1. MapReduce (defines): framework to define a data processing task
2. YARN (runs): framework to run the data processing tasks

# Intro

### HDFS
Hadoop is deployed on a group of machine (refered as cluster). Each machine in the cluster is called as node.
* **Name node**: acts as mster node. manages overall file system.
    * It stores the directory structure
    * Metadata for all the files (block locations of each file)
* **Data node**: data is physically stored in name nodes

For example a file is broken into blocks of size 128 MB.
**How is the size choosed?**
Aimed to minimize the time to seek the block on the disk. Once seeked we could read the data. 

**Steps on reading a File**

1. The metadata in name node (its referred first)
2. The blocks in the data node

**What if one of the blocks get corrupted or what happens when a node fail?**
Solution: replication factor. Copy the blocks in different node.

### MapReduce

**Find the number of time hello has occured in the file**

1. **Map Phase**: Process each block in the node that its stored and get intermediate results.
2. **Reduce Phase**: Take all the results to one node and combine them.

### YARN

**Yet another resource negotiator**
1. Resposible for the management of resources on the Hadoop cluster.
2. Monitor failures

### Different modes
1. Standalone mode: only one node (technically no HDFS, no YARN)
2. Pseudo-distributed: two nodes are running,one acts as a name node and other as a data node
3. Fully distributed: Manually configuring and managing nodes in a Hadoop cluster is complicated and error-prone. We could use enterprise edition of Hadoop: Cloudera, Hortonworks

## MapReduce

Any data processing task can be parallelized if we express it as 
<key, value> --> map() --> <key, value> --> reduce()

We could also chain transformations.

**Once the results from all the mapper come, it stored on a single node. In this node an operation called sort/merge occurs.**

## Parallelize your reduce operation

### Combiner function
(reducer applied at each data node)

Map operation is highly parallelized. Example: constitutes to the number of data nodes which actually contains the blocks of data.

Instead of sending the mapper results to a single node to perform sort/merge we could use the combiner function to merge/combine the key value pairs with the same keys.

The combiner function is implemented by extending the reducer class.

**IMP**
The combiner makes the MapReduce task faster by 
1. Reduce operation is partially parallelized
2. Less data needs to be transferred across the network

### Constraints on combiner function

NOTE: The combiner function should have no impact on the final result. In fact, the decision of whether to use the combiner or not ultimately rests with Hadoop.
For example: Average is a reducer function, if we use it as a combiner function we might end up getting wrong results.
```
Data node 1 --> MAP --> 
Data node 1 --> MAP --> Sort/merge --> Reduce
Data node 1 --> MAP -->

```

Average of a set of number <> Average (Average of subset of numbers)





### Shuffle and Sort

(Allows us to use multiple reducer)

**How is a map task excuted?**
Is executed by a "process" on a node in the cluster. A process is a job which has its own memory and manages its own resources.

All the processes are launched and managed by YARN.
- Map processes are called as mappers
- Reduce processes are called as reducers

**How many mappers and reducers does a MapReduce job require?**

**Mappers**
- The number of mappers are usually controlled by YARN. Each mapper operates on one HDFS block (128 MB). The number of mappers depends on the **number of splits** i.e., the number of blocks your input file is broken into. This is number which YARN uses.
- This effectively means that multiple mappers can be assigned to 1 data node. (This is in reality)
- The number of mappers can only be controlled to a limited extent by the user.

**Reducers**
- The number of reducers can be controlled by users
- By default, there is only 1 reducer (which is what we have seen so far)
- With multiple reducers, the reduce task can also be parallelized

The number of reducers can be specified when we run the job.

**Scenario**
- Let's say our input file has been split into 4 blocks by HDFS.There would be 4 mappers.
<img src="./Image/image1.png">

When there are 2 reducers, the output of each map is first partitioned into 2 partitions. 

**How do we partition our map output?**
The map output is assigned to a partition based on a key. All the values with the same key must be assigned to the same partition.

The output of partition 1 --shuffled and sort--> reducer1 and partition 2 --shuffled and sor--> reducer 2

Partitioning and Shuffle/Sort are 2 extra operations that happens for MR jobs with multiple reducers.

<img src="./Image/image2.png">


`Number of partitions = Number of reducers`

**Hadoop taskes care of mapping keys to partition**
The partition number of key depends on it hash value.
Partition logic should be such that the keys are evenly distributed among the partitions

## Hadoop Streaming API

We can implement the map() and reduce() functions in ANY LANGUAGE. The streaming API uses standard INP/OUT inorder to communicate with our program.


## HDFS

Hadoop is normally deployed on a group of machines. This group of machines is called as cluster. Every machine in the cluster is a node.

**Name Node**
One of the node acts as the master node, which manages the overall file system. It store
1. Directory structure
2. Metadata for all the files

**Data Nodes**
Other nodes are called data nodes. The data is physically stored on these nodes. 

**Scenario: Files stored in HDFS**
- First the file is broken into blocks of size 128 MB
(The size is chosen to minimize the time to seek to the block on the disk)
- These blocks are then stored accross data nodes. A data node can contains more than one blocks

**What if one of the block or node gets crashed?**
Define replication factor in HDFS. It is defined in configuration file `hdfs-site.xml`.

In prod usually the replication factor is 3.

**The replica locations are chosen to maximize redundancy while minimizing write bandwidth.**
- To **maximize redundancy** all replicas should be sored "far away" i.e., different racks.

* **minimize the write bandwidth**
    * When a file is written to HDFS, a single node runs the replication pipeline. 
    * A node is chosen and this node forwards the data to the location chosen for the second replica. (First within the rack then across the rack).
    * All this consumes network bandwidth. More the number of racks to traverse, more the bandwidth consumed.
    
### Default Strategy
For a replication factor of 3
1. First replica is chosen in the same rack but different node
2. Second replica is chosen in a different rack (rack 2) and stored in node 1
3. Third replica is chosen in rack 2 but different node 2
